In [33]:
import findspark
import os
import tempfile
findspark.init()
from pyspark.sql import SparkSession, types

In [34]:
spark = SparkSession.builder.appName('Pengelolaan Sampah').getOrCreate()

In [35]:
df = spark.read.csv('Downloads/tempat-penampungan-sampah-sementara-tps-di-wilayah-dki-jakarta-tahun-2019.csv', header=True, inferSchema=True)
df.count()
# df.show()

1125

In [36]:
df = df.na.drop()
df.count()

923

In [37]:
# df.write.csv('Downloads/data.csv', header=True)

In [38]:
df = df.drop("nama_tps", "kelurahan", "alamat", "kecamatan", "status_lahan")

In [39]:
df.filter("volume_sampah_rata_rata_per_hari_m3>=1000").show()

+-------------+--------------+-------------+-----------------------------------+
|      wilayah|     jenis_tps|luas_lahan_m2|volume_sampah_rata_rata_per_hari_m3|
+-------------+--------------+-------------+-----------------------------------+
|Jakarta Barat|  Pool Gerobak|           80|                              24255|
|Jakarta Barat|  Pool Gerobak|          350|                              77625|
|Jakarta Barat|  Pool Gerobak|          250|                               7785|
|Jakarta Barat|  Pool Gerobak|          250|                              54765|
|Jakarta Barat|     Bak Beton|           33|                              16245|
|Jakarta Barat|Pool Container|           60|                               4455|
|Jakarta Barat|Pool Container|           42|                               6435|
|Jakarta Barat|Pool Container|           50|                               7875|
|Jakarta Barat|  Pool Gerobak|           50|                               6435|
|Jakarta Barat|  Pool Geroba

In [40]:
df.printSchema()

root
 |-- wilayah: string (nullable = true)
 |-- jenis_tps: string (nullable = true)
 |-- luas_lahan_m2: string (nullable = true)
 |-- volume_sampah_rata_rata_per_hari_m3: string (nullable = true)



In [41]:
df = df.withColumn("luas_lahan_m2", df["luas_lahan_m2"].cast(types.FloatType()))
df = df.withColumn("volume_sampah_rata_rata_per_hari_m3", df["volume_sampah_rata_rata_per_hari_m3"].cast(types.FloatType()))
df.printSchema()

root
 |-- wilayah: string (nullable = true)
 |-- jenis_tps: string (nullable = true)
 |-- luas_lahan_m2: float (nullable = true)
 |-- volume_sampah_rata_rata_per_hari_m3: float (nullable = true)



In [42]:
df.groupBy("wilayah").sum("luas_lahan_m2").show()

+---------------+------------------+
|        wilayah|sum(luas_lahan_m2)|
+---------------+------------------+
|Jakarta Selatan|            5702.0|
|  Jakarta Utara|           8312.25|
|  Jakarta Barat|           23049.0|
|  Jakarta Pusat|            8528.0|
| Jakarta Timur |27395.100000023842|
+---------------+------------------+



In [43]:
df = df.withColumn("perbandingan", df["luas_lahan_m2"]/df["volume_sampah_rata_rata_per_hari_m3"])
df.show()

+-------------+---------+-------------+-----------------------------------+------------------+
|      wilayah|jenis_tps|luas_lahan_m2|volume_sampah_rata_rata_per_hari_m3|      perbandingan|
+-------------+---------+-------------+-----------------------------------+------------------+
|Jakarta Pusat|Bak Beton|         14.0|                                1.0|              14.0|
|Jakarta Pusat|Bak Beton|         60.0|                                8.0|               7.5|
|Jakarta Pusat|Bak Beton|         18.0|                                8.0|              2.25|
|Jakarta Pusat|Bak Beton|         20.0|                                6.0|3.3333333333333335|
|Jakarta Pusat|Bak Beton|          8.0|                                1.0|               8.0|
|Jakarta Pusat|Bak Beton|         12.0|                                2.0|               6.0|
|Jakarta Pusat|Bak Beton|         15.0|                                1.0|              15.0|
|Jakarta Pusat|Bak Beton|         24.0|           

In [44]:
df.groupBy("wilayah").sum("perbandingan").show()

+---------------+------------------+
|        wilayah| sum(perbandingan)|
+---------------+------------------+
|Jakarta Selatan| 907.0611111111111|
|  Jakarta Utara| 673.3426497800738|
|  Jakarta Barat|1848.4403453328052|
|  Jakarta Pusat|1721.2011649870294|
| Jakarta Timur |4408.9903434170255|
+---------------+------------------+



In [45]:
df.printSchema()

root
 |-- wilayah: string (nullable = true)
 |-- jenis_tps: string (nullable = true)
 |-- luas_lahan_m2: float (nullable = true)
 |-- volume_sampah_rata_rata_per_hari_m3: float (nullable = true)
 |-- perbandingan: double (nullable = true)

